In [1]:
%%time

import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.linear_model import LinearRegression

# Función para resetear columnas

def reset_columns(df):
    df.columns = ['_'.join(col) for col in df.columns.values]
    return df


# Función para ponderar a total población - individuos en millones las variables de condiciones de vida y otras
def weights(dfw,weight_varlist,factor):
    for var in weight_varlist:
        dfw[var] = dfw[var] * dfw[factor]
    return dfw

df = pd.read_csv('./Files/ECV_2004_2018.csv.gz', compression = 'gzip', sep = ';')
df2 = df.copy()

# Selección de las variables necesarias para visualización y para el modelo
df2 = df2[(df2['Year'] == 2013) | (df2['Year'] == 2018) |
      (df2['Year'] == 2008) | (df2['Year'] == 2009) |
      (df2['Year'] == 2010) | (df2['Year'] == 2011) |
      (df2['Year'] == 2012) | (df2['Year'] == 2013) |
      (df2['Year'] == 2014) | (df2['Year'] == 2015) |
      (df2['Year'] == 2016) | (df2['Year'] == 2017)][['Year','Year_HHid','HHId', 'HHHolidays','HHFood','HHReserves', 'HHComputer', 'HHCar',
                                                     'AREMonth', 'HousingCost', 'HHHeath', 'vhRentaa', 'vhPobreza', 'vhMATDEP',
                                                     'Media_renta', 'vhRentaa_zscore', 'vhRentaa_xperson', 'Habitat', 'Comunidad Autónoma_x',
                                                     'PIB_', 'Pob_', 'PIB_percapita_', 'PIB(%)_change_', 'PIB(%)_varnacional_', 'PIB_Nacional',
                                                     'Pob_Nacional', 'PIB_percapita_Nacional', 'PIB(%)_change_Nacional', 'WorStatus',
                                                     'FStudies', 'CrConditions', 'HLimitations', 'MDClothes', 'MDShoes',
                                                     'MDFriends', 'MDLeisure', 'MDSelf', 'MDInternet','HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                                     'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                                     'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                                     'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                                     'CHealth','HousingCost_LowImpactHH', 'HLimitations_NoSerLimitedG_limitado', 'vhMATDEP_vhMATDEP_No', 'MDClothes_Yes', 
                                                     'HHFood_No', 'MDShoes_Yes', 'MDShoes_No affordable', 'MDLeisure_Yes', 
                                                      'CrConditions_YChronic', 'HHWashMachine_No', 'HHWashMachine_Yes', 
                                                      'MDFriends_Yes', 'MDFriends_No affordable', 'MDInternet_Yes', 
                                                      'MDClothes_No affordable', 'HLimitations_SerLimited', 'HHCar_No', 
                                                      'HousingCost_HighImpactHH', 'MDLeisure_No affordable', 'HHHeath_No', 
                                                      'vhPobreza_vhPobreza_No', 'MDInternet_No affordable', 
                                                      'MDSelf_No affordable', 'HHComputer_No', 'HHHolidays_No', 
                                                      'HHReserves_No', 'MDSelf_Yes','Region','AgeNumeric']].dropna()

# Estimación de la felicidad
df_model = pd.read_csv('./Files/ECV_2004&2018.csv.gz', compression = 'gzip', sep = ';')

# Meter el ID
df_model = df_model[['Year','Year_HHid' ,'HHHolidays_Yes','HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth','LifeSatisfaction2','HousingCost_LowImpactHH', 
                                      'HLimitations_NoSerLimitedG_limitado', 'vhMATDEP_vhMATDEP_No', 'MDClothes_Yes', 
                                      'HHFood_No', 'MDShoes_Yes', 'MDShoes_No affordable', 'MDLeisure_Yes', 
                                      'CrConditions_YChronic', 'HHWashMachine_No', 'HHWashMachine_Yes', 'MDFriends_Yes', 
                                      'MDFriends_No affordable', 'MDInternet_Yes', 'MDClothes_No affordable', 
                                      'HLimitations_SerLimited', 'HHCar_No', 'HousingCost_HighImpactHH', 
                                      'MDLeisure_No affordable', 'WorStatus_EmployeeFT', 'WorStatus_Unemployed', 
                                      'HHHeath_No', 'WorStatus_Student', 'FStudies_EdSuperior', 'vhPobreza_vhPobreza_No', 
                                      'FStudies_<Primaria', 'MDInternet_No affordable', 'MDSelf_No affordable', 
                                      'HHComputer_No', 'HHHolidays_No', 'HHReserves_No', 'MDSelf_Yes','Region','Pob_',
                                      'PIB_percapita_','AgeNumeric']].dropna()

# Variables modelo predicciones 2008 a 2018
X = df_model[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth']]
y = df_model['LifeSatisfaction2']


reg2 = LinearRegression()
reg2.fit(X, y)
# Predicciones
df2['Predictions'] =reg2.predict(df2[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth']])

# Cambio nombre variable
df2.rename(columns = {'PIB(%)_change_Nacional': 'PIB_change_Nacional'}, inplace = True) 
# Ponderación año
df_weigths_nac = (df2.groupby(['Year'])['Pob_Nacional'].mean() \
             / df2.groupby(['Year'])['HHId'].count()).reset_index().rename(columns = {0:'Weight'})
# Incluida ponderación por año en el df
df2 = df2.merge(df_weigths_nac)

# Ponderación año y región
df_weigths_reg = (df2.groupby(['Year','Region'])['Pob_'].mean() \
           / df2.groupby(['Year','Region'])['HHId'].count()).reset_index().rename(columns = {0:'Weight_reg'})
# Incluida ponderación por año en el df
df2 = df_weigths_reg.merge(df2, left_on = ['Year','Region'], right_on = ['Year','Region'])
# Creación de quintiles con las predicciones
df2['Quintiles_bins'] = pd.qcut(df2['Predictions'],5) # Distribuir todas las observaciones predecidas en 5 quintiles

# Asignación de quintil según los rangos resultantes
df2['Quintiles'] = df2['Quintiles_bins'].apply(lambda x: '1º - Inferior' if str(x) == '(3.6550000000000002, 6.297]' \
                                                       else '2º' if str(x) ==  '(6.297, 6.875]' \
                                                        else '3º' if str(x) ==  '(6.875, 7.346]' \
                                                        else '4º' if str(x) ==  '(7.346, 7.728]' \
                                                        else '5º - Superior')

# dfs para data set de visualización centrado en el contexto, datos globales de economía y 

df_context_sum = df2.groupby(['Year','Quintiles'])['Weight',
                            'PIB_change_Nacional'].sum().reset_index()
df_context_mean = df2.groupby(['Year'])[['Predictions','PIB_percapita_Nacional','AgeNumeric']].mean().reset_index()
df_context =df_context_sum.merge(df_context_mean)

df_context_sum_reg = df2.groupby(['Year','Region','Quintiles'])['Weight_reg'].sum().reset_index()
df_context_mean_reg = df2.groupby(['Year','Region'])[['Predictions','PIB_percapita_','AgeNumeric']].mean().reset_index()
df_context_reg =df_context_sum_reg.merge(df_context_mean_reg)


# Variables en la matriz de correlaciones y otros 
var_corr = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes','LifeSatisfaction2']


df_weights_nac = df2.copy()
df_weights_reg = df2.copy()
df_means_nac = df2.copy()
df_means_reg = df2.copy()
df_counts_nac = df2.copy()
df_counts_reg = df2.copy()

var_list2 = var_corr[:-1]


df_weights_nac = weights(df_weights_nac ,var_list2,'Weight')
df_means_nac = weights(df_means_nac,var_list2,'Predictions') # Para qué he puesto aquí la media?

df_weights_reg = weights(df_weights_reg,var_list2,'Weight_reg')
df_means_reg = weights(df_means_reg,var_list2,'Predictions')

var_list3 = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes']

var_list4 = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes','Year']

dictionary_map = {'LifeSatisfaction2':'Felicidad',
                         'HHHolidays_Yes':'Vacaciones Sí',
                         'HHFood_Yes':'Alimentación Sí',
                         'HHReserves_Yes':'"Colchón" económico Sí',
                         'HHReserves_No':'"Colchón" económico No',
                         'HHCar_Yes':'Coche Sí',
                         'HHHeath_Yes':'Calefacción Sí',
                         'vhPobreza_vhPobreza_No':'Riesgo de pobreza No',
                         'vhMATDEP_vhMATDEP_No':'Carencia Material Severa No',
                         'MDClothes_Yes': 'Compra de ropa Sí',
                         'MDShoes_Yes':'Compra de zapatos Sí',
                         'MDFriends_Yes': 'Ocio con amigos Sí',
                         'MDLeisure_Yes': 'Ocio en general Sí',
                         'MDSelf_Yes':'Gasto en uno mismo Sí',
                         'MDInternet_Yes': 'Acceso a internet Sí',
                         'HHHolidays_No':'Vacaciones No', 
                         'HHFood_No':'Alimentación No', 
                         'HHComputer_No':'Ordenador No', 
                         'HHComputer_Yes':'Ordenador Sí',
                         'HHWashMachine_No':'Lavadora Sí',
                         'HHWashMachine_Yes':'Lavadora No', 
                         'HHCar_No': 'Coche No',
                         'HousingCost_HighImpactHH':'Alto impacto coste vivienda', 
                         'HousingCost_LowImpactHH':'Bajo impacto coste vivienda',
                         'HousingCost_MediumImpactHH':'Impacto medio coste vivienda',
                         'HHHeath_No':'Calefacción No', 
                         'vhPobreza_vhPobreza_Yes': 'Riesgo de pobreza Sí', 
                         'vhMATDEP_vhMATDEP_Yes': 'Carencia material severa',
                         'CrConditions_NChronic': 'Enfermedades Crónicas No',
                         'CrConditions_YChronic':'Enfermedades Crónicas Sí', 
                         'HLimitations_NoLimited': 'Limitaciones físicas No',
                         'HLimitations_NoSerLimitedG_limitado' : 'Limitaciones físicas leves Sí', 
                         'HLimitations_SerLimited': 'Limitaciones físicas graves Sí',
                         'MDClothes_No affordable' : 'Compra de ropa No',
                         'MDShoes_No affordable' : 'Compra de zapatos No',
                         'MDFriends_No affordable': 'Ocio con amigos No',
                         'MDLeisure_No affordable': 'Ocio en general No',
                         'MDSelf_No affordable': 'Gasto en uno mismo No',
                         'MDInternet_No affordable': 'Acceso a internet No'}


# Suma de condiciones sociales por el peso que tienen en la población real

# Nac
# Sumamos el valor de las dumies que se les ha aplicado un valor de ponderación por individuo
df_grouped_sum_nac = df_weights_nac.groupby('Year')[var_list3].sum().reset_index()
# Cambio de estructura de columnas a filas
df_grouped2_sum_m_nac = pd.melt(df_grouped_sum_nac, id_vars=['Year'], value_vars = var_list3) # ¿Qué he hecho aquí?
# Cambio de nombre de las variables por otro más fácil de indentificar para la visualización
df_grouped2_sum_m_nac['variable'] = df_grouped2_sum_m_nac['variable'].map(dictionary_map)

# Reg
# Sumamos el valor de las dumies que se les ha aplicado un valor de ponderación por individuo 
df_grouped_sum_reg = df_weights_reg.groupby(['Year','Region'])[var_list3].sum().reset_index()
# Cambio de estructura de columnas a filas
df_grouped2_sum_m_reg = pd.melt(df_grouped_sum_reg, id_vars=['Year','Region'], value_vars = var_list3) # ¿Qué he hecho aquí?
# Cambio de nombre de las variables por otro más fácil de indentificar para la visualización
df_grouped2_sum_m_reg['variable'] = df_grouped2_sum_m_reg['variable'].map(dictionary_map)

# Media de la satisfacción vital / Felicidad para cada una de esas condiciones

# Nac
# Sumamos las predicciones totales obtenidas de felicidad
df_grouped_mean_nac = df_means_nac.groupby('Year')[var_list3].sum().reset_index()
# Elevar el año al 2 para dividir posteriormente el dataset
df_grouped_mean_nac['Year'] = df_grouped_mean_nac['Year']**2 # Revisar si necesito las medias
# Sumamos el número total de individuos de la muestra asociados a cada característica
df_grouped_counts_nac = df_counts_nac.groupby('Year')[var_list3].sum().reset_index()
# Dividimos el número total de puntos de felicidad obtenidos entre el número de pesonas - queda una media asociada a cada característica
df_grouped_mean_c_nac = df_grouped_mean_nac / df_grouped_counts_nac
# Transformación del df columnas por filas
df_grouped2_mean_m_nac = pd.melt(df_grouped_mean_c_nac, id_vars=['Year'], value_vars = var_list3)     
# Recodificamos las variables
df_grouped2_mean_m_nac['variable'] = df_grouped2_mean_m_nac['variable'].map(dictionary_map)

# Media de la satisfacción vital / Felicidad para cada una de esas condiciones

# Reg
# Sumamos las predicciones totales obtenidas de felicidad
df_grouped_mean_reg = df_means_reg.groupby(['Year','Region'])[var_list3].sum().reset_index()
# Elevar el año al 2 para dividir posteriormente el df
df_grouped_mean_reg['Year'] = df_grouped_mean_reg['Year']**2 # Revisar si necesito las medias
# Sumamos el número total de individuos de la muestra asociados a cada característica
df_grouped_counts_reg = df_counts_reg.groupby(['Year','Region'])[var_list3].sum().reset_index()
# Dividimos el número total de puntos de felicidad obtenidos entre el número de pesonas - queda una media asociada a cada característica
df_grouped_mean_c_reg = df_grouped_mean_reg.copy()
df_grouped_mean_c_reg[var_list4] = df_grouped_mean_reg[var_list4] / df_grouped_counts_reg[var_list4]
# Transformación del df columnas por filas
df_grouped2_mean_m_reg = pd.melt(df_grouped_mean_c_reg, id_vars=['Year','Region'], value_vars = var_list3)     
# Recodificamos las variables
df_grouped2_mean_m_reg['variable'] = df_grouped2_mean_m_reg['variable'].map(dictionary_map)

# df con ambos valores -nacional
df_conditions_nac = df_grouped2_sum_m_nac.merge(df_grouped2_mean_m_nac, right_on = ['Year','variable'], 
                                        left_on = ['Year','variable']).rename(columns = {'value_x': 'Ind_conditions',
                                                                                        'value_y': 'Mean_conditions'})

# df con ambos valores - regional
df_conditions_reg = df_grouped2_sum_m_reg.merge(df_grouped2_mean_m_reg, right_on = ['Year','variable','Region'], 
                                        left_on = ['Year','variable','Region']).rename(columns = {'value_x': 'Ind_conditions',
                                                                                        'value_y': 'Mean_conditions'})


dictionary_variables = {'Vacaciones No': 'Vacaciones', 
                        'Vacaciones Sí': 'Vacaciones',
                        'Alimentación No': 'Alimentación',
                        'Alimentación Sí': 'Alimentación',
                        '"Colchón" económico Sí': '"Colchón" económico',
                        '"Colchón" económico No': '"Colchón" económico',
                        'Ordenador No': 'Ordenador',
                        'Ordenador Sí': 'Ordenador',
                        'Lavadora Sí': 'Lavadora',
                        'Lavadora No': 'Lavadora', 
                        'Coche No' : 'Coche',
                        'Coche Sí' : 'Coche',
                        'Alto impacto coste vivienda': 'Vivienda',
                        'Bajo impacto coste vivienda': 'Vivienda',
                        'Impacto medio coste vivienda': 'Vivienda',
                        'Calefacción No': 'Calefacción', 
                        'Calefacción Sí': 'Calefacción', 
                        'Riesgo de pobreza No': 'Riesgo de pobreza',
                        'Riesgo de pobreza Sí': 'Riesgo de pobreza',
                        'Carencia Material Severa No': 'Carencia material',
                        'Carencia material severa': 'Carencia material',
                        'Enfermedades Crónicas No': 'Enfermedades Crónicas',
                        'Enfermedades Crónicas Sí': 'Enfermedades Crónicas',
                        'Limitaciones físicas No': 'Limitaciones Físicas',
                        'Limitaciones físicas leves Sí': 'Limitaciones Físicas',
                        'Limitaciones físicas graves Sí': 'Limitaciones Físicas',
                        'Compra de ropa No': 'Compra de ropa',
                        'Compra de ropa Sí': 'Compra de ropa',
                        'Compra de zapatos No': 'Compra de zapatos',
                        'Compra de zapatos Sí': 'Compra de zapatos',
                        'Ocio con amigos No': 'Ocio con amigos',
                        'Ocio con amigos Sí': 'Ocio con amigos',
                        'Ocio en general No': 'Ocio General',
                        'Ocio en general Sí': 'Ocio General',
                        'Gasto en uno mismo No': 'Gasto en uno mismo', 
                        'Gasto en uno mismo Sí': 'Gasto en uno mismo',
                        'Acceso a internet No': 'Internet', 
                        'Acceso a internet Sí': 'Internet'}



# Categorías
dictionary_category=  {'Vacaciones No': 'Economía no básica', 
                        'Vacaciones Sí': 'Economía no básica',
                        'Alimentación No': 'Economía básica',
                        'Alimentación Sí': 'Economía básica',
                        '"Colchón" económico Sí': 'Situación económica',
                        '"Colchón" económico No': 'Situación económica',
                        'Ordenador No': 'Economía no básica',
                        'Ordenador Sí': 'Economía no básica',
                        'Lavadora Sí': 'Economía básica',
                        'Lavadora No': 'Economía básica', 
                        'Coche No' : 'Economía no básica',
                        'Coche Sí' : 'Economía no básica',
                        'Alto impacto coste vivienda': 'Situación económica',
                        'Bajo impacto coste vivienda': 'Situación económica',
                        'Impacto medio coste vivienda': 'Situación económica',
                        'Calefacción No': 'Economía básica', 
                        'Calefacción Sí': 'Economía básica', 
                        'Riesgo de pobreza No': 'Situación económica',
                        'Riesgo de pobreza Sí': 'Situación económica',
                        'Carencia Material Severa No': 'Situación económica',
                        'Carencia material severa': 'Situación económica',
                        'Enfermedades Crónicas No': 'Salud',
                        'Enfermedades Crónicas Sí': 'Salud',
                        'Limitaciones físicas No': 'Salud',
                        'Limitaciones físicas leves Sí': 'Salud',
                        'Limitaciones físicas graves Sí': 'Salud',
                        'Compra de ropa No': 'Economía básica',
                        'Compra de ropa Sí': 'Economía básica',
                        'Compra de zapatos No':'Economía básica',
                        'Compra de zapatos Sí': 'Economía básica',
                        'Ocio con amigos No': 'Economía no básica',
                        'Ocio con amigos Sí': 'Economía no básica',
                        'Ocio en general No': 'Economía no básica',
                        'Ocio en general Sí': 'Economía no básica',
                        'Gasto en uno mismo No': 'Economía no básica', 
                        'Gasto en uno mismo Sí': 'Economía no básica',
                        'Acceso a internet No': 'Economía no básica', 
                        'Acceso a internet Sí': 'Economía no básica'}

# Respuesta
dictionary_response =  {'Vacaciones No': 'No', 
                        'Vacaciones Sí': 'Sí',
                        'Alimentación No': 'No',
                        'Alimentación Sí': 'Sí',
                        '"Colchón" económico Sí': 'Sí',
                        '"Colchón" económico No': 'No',
                        'Ordenador No': 'No',
                        'Ordenador Sí': 'Sí',
                        'Lavadora Sí': 'Sí',
                        'Lavadora No': 'No', 
                        'Coche No' : 'No',
                        'Coche Sí' : 'Sí',
                        'Alto impacto coste vivienda': 'Impacto Alto',
                        'Bajo impacto coste vivienda': 'Impacto Bajo',
                        'Impacto medio coste vivienda': 'Impacto Medio',
                        'Calefacción No': 'No', 
                        'Calefacción Sí': 'Sí', 
                        'Riesgo de pobreza No': 'No',
                        'Riesgo de pobreza Sí': 'Sí',
                        'Carencia Material Severa No': 'No',
                        'Carencia material severa': 'Sí',
                        'Enfermedades Crónicas No': 'No',
                        'Enfermedades Crónicas Sí': 'Sí',
                        'Limitaciones físicas No': 'No',
                        'Limitaciones físicas leves Sí':  'Sí leves',
                        'Limitaciones físicas graves Sí':  'Sí graves',
                        'Compra de ropa No': 'No',
                        'Compra de ropa Sí':  'Sí',
                        'Compra de zapatos No':'No',
                        'Compra de zapatos Sí':  'Sí',
                        'Ocio con amigos No': 'No',
                        'Ocio con amigos Sí':  'Sí',
                        'Ocio en general No': 'No',
                        'Ocio en general Sí':  'Sí',
                        'Gasto en uno mismo No': 'No', 
                        'Gasto en uno mismo Sí':  'Sí',
                        'Acceso a internet No': 'No', 
                        'Acceso a internet Sí': 'Sí'}

# Incidencia positiva - negativa (comprobado con el coeficiente de correlación de cada una de las variables con la felicidad
dictionaryp_posneg  =  {'Vacaciones No': 'Negativo', 
                        'Vacaciones Sí': 'Positivo',
                        'Alimentación No': 'Negativo',
                        'Alimentación Sí': 'Positivo',
                        '"Colchón" económico Sí': 'Positivo',
                        '"Colchón" económico No': 'Negativo',
                        'Ordenador No': 'Negativo',
                        'Ordenador Sí': 'Positivo',
                        'Lavadora Sí': 'Positivo',
                        'Lavadora No': 'Negativo', 
                        'Coche No' : 'Negativo',
                        'Coche Sí' : 'Positivo',
                        'Alto impacto coste vivienda': 'Negativo',
                        'Bajo impacto coste vivienda': 'Positivo',
                        'Impacto medio coste vivienda': 'Positivo',
                        'Calefacción No': 'Negativo', 
                        'Calefacción Sí': 'Positivo', 
                        'Riesgo de pobreza No': 'Positivo',
                        'Riesgo de pobreza Sí': 'Negativo',
                        'Carencia Material Severa No': 'Positivo',
                        'Carencia material severa': 'Negativo',
                        'Enfermedades Crónicas No': 'Positivo',
                        'Enfermedades Crónicas Sí': 'Negativo',
                        'Limitaciones físicas No': 'Positivo',
                        'Limitaciones físicas leves Sí': 'Negativo',
                        'Limitaciones físicas graves Sí':  'Negativo',
                        'Compra de ropa No': 'Negativo',
                        'Compra de ropa Sí': 'Positivo',
                        'Compra de zapatos No':'Negativo',
                        'Compra de zapatos Sí':  'Positivo',
                        'Ocio con amigos No': 'Negativo',
                        'Ocio con amigos Sí':  'Positivo',
                        'Ocio en general No': 'Negativo',
                        'Ocio en general Sí':  'Positivo',
                        'Gasto en uno mismo No': 'Negativo', 
                        'Gasto en uno mismo Sí':  'Positivo',
                        'Acceso a internet No': 'Negativo', 
                        'Acceso a internet Sí': 'Positivo'}



# Codificación de variables - nacional
df_conditions_nac['variable_name'] = df_conditions_nac['variable'].map(dictionary_variables)
df_conditions_nac['variable_response'] = df_conditions_nac['variable'].map(dictionary_response)
df_conditions_nac['variable_category'] = df_conditions_nac['variable'].map(dictionary_category)
df_conditions_nac['variable_posneg'] = df_conditions_nac['variable'].map(dictionaryp_posneg)


# Codificación de variables - nacional
df_conditions_reg['variable_name'] = df_conditions_reg['variable'].map(dictionary_variables)
df_conditions_reg['variable_response'] = df_conditions_reg['variable'].map(dictionary_response)
df_conditions_reg['variable_category'] = df_conditions_reg['variable'].map(dictionary_category)
df_conditions_reg['variable_posneg'] = df_conditions_reg['variable'].map(dictionaryp_posneg)

# DFs con la población y el año 

df_Pob_nac = df2.groupby(['Year'])['Pob_Nacional'].mean().reset_index()
df_Pob_reg = df2.groupby(['Year','Region'])['Pob_'].mean().reset_index()


# Merge de contexto, condiciones y población 
# nacional


df_nac_trial = (df_conditions_nac.merge(df_Pob_nac, left_on = ['Year'], right_on = ['Year'])).merge(df_context, \
        left_on = ['Year'], right_on = ['Year'])   
# regional
df_regs_trial = (df_conditions_reg.merge(df_Pob_reg, left_on = ['Year','Region'], right_on = ['Year','Region'])).merge(df_context_reg, \
        left_on = ['Year','Region'], right_on = ['Year','Region'])      

# Cálculo proporciones nac
df_nac_trial['Ind_conditions_prop'] = df_nac_trial['Ind_conditions'] / df_nac_trial['Pob_Nacional']
df_nac_trial['Ind_condition_quint_prop'] = df_nac_trial['Weight'] / df_nac_trial['Pob_Nacional'] 

# Cálculo proporciones reg
df_regs_trial['Ind_conditions_prop'] = df_regs_trial['Ind_conditions'] / df_regs_trial['Pob_']
df_regs_trial['Ind_condition_quint_prop'] = df_regs_trial['Weight_reg'] / df_regs_trial['Pob_'] 


#df_regs_trial.to_csv('./Files/regions_visualization.csv', sep = ';')
#df_nac_trial.to_csv('./Files/nac_visualization.csv', sep = ';')

/home/mariacarrasco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:114: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


CPU times: user 11.1 s, sys: 14.3 s, total: 25.5 s
Wall time: 23 s


In [45]:
# Corrección de nombres de variables
def rename(obs):
    if obs == 'Alto impacto coste vivienda':
        return 'Impacto coste vivienda alto'
    elif obs == 'Bajo impacto coste vivienda':
        return 'Impacto coste vivienda bajo'
    elif obs == 'Impacto medio coste vivienda':
        return 'Impacto coste vivienda medio'
    elif obs == 'Ocio con amigos Sí':
        return 'Ocio con amigos Sí - 2009 & desde 2013'
    elif obs == 'Ocio en general Sí':
        return 'Ocio en general Sí - 2009 & desde 2013'         
    elif obs == 'Gasto en uno mismo Sí':
        return 'Gasto en uno mismo Sí- 2009 & desde 2013'   
    elif obs == 'Acceso a internet Sí':
        return 'Acceso a internet Sí - 2009 & desde 2013'
    else:
        return obs
    
df_regs_trial['variable_renamed'] = df_regs_trial['variable'].apply(lambda x: rename(x))
df_nac_trial['variable_renamed'] = df_nac_trial['variable'].apply(lambda x: rename(x))

df_regs_trial.to_csv('./Files/regions_visualization_v2.csv', sep = ';')
df_nac_trial.to_csv('./Files/nac_visualization_v2.csv', sep = ';')

In [64]:
df_regs_trial.head()

,Year,Region,variable,Ind_conditions,Mean_conditions,variable_name,variable_response,variable_category,variable_posneg,Pob_,Quintiles,Weight_reg,Predictions,PIB_percapita_,AgeNumeric,Ind_conditions_prop,Ind_condition_quint_prop,variable_renamed
0,2008,Andalucía,Vacaciones No,4.385302e+06,6.290069,Vacaciones,No,Economía no básica,Negativo,8202220,1º - Inferior,2.124882e+06,6.82002,18.437307,47.215618,0.534648,0.259062,Vacaciones No
1,2008,Andalucía,Vacaciones No,4.385302e+06,6.290069,Vacaciones,No,Economía no básica,Negativo,8202220,2º,2.037438e+06,6.82002,18.437307,47.215618,0.534648,0.248401,Vacaciones No
2,2008,Andalucía,Vacaciones No,4.385302e+06,6.290069,Vacaciones,No,Economía no básica,Negativo,8202220,3º,1.709525e+06,6.82002,18.437307,47.215618,0.534648,0.208422,Vacaciones No
3,2008,Andalucía,Vacaciones No,4.385302e+06,6.290069,Vacaciones,No,Economía no básica,Negativo,8202220,4º,1.180490e+06,6.82002,18.437307,47.215618,0.534648,0.143923,Vacaciones No
4,2008,Andalucía,Vacaciones No,4.385302e+06,6.290069,Vacaciones,No,Economía no básica,Negativo,8202220,5º - Superior,1.149885e+06,6.82002,18.437307,47.215618,0.534648,0.140192,Vacaciones No


In [58]:
df_regs_trial2 = df_regs_trial.copy()
df_regs_trial2_1 = pd.melt(df_regs_trial.copy(), id_vars=['Year','Region','PIB_percapita_','Quintiles'],\
                           value_vars = ['Ind_condition_quint_prop'])
df_regs_trial2_1 = df_regs_trial2_1[df_regs_trial2_1['Quintiles'] == '5º - Superior']
df_regs_trial2_1.drop_duplicates()
df_regs_trial_3 = df_regs_trial.groupby(['Year','Region','variable_renamed'])['Ind_conditions_prop'].mean().reset_index()
df_regs_final = df_regs_trial2_1.merge(df_regs_trial_3, right_on = ['Year','Region'], left_on = ['Year','Region'])
df_regs_final.to_csv('./Files/regions_visualization_v2_grouped.csv', sep = ';')
df2.groupby('Year')['PIB_change_Nacional','PIB_percapita_Nacional'].mean().reset_index().to_csv('./Files/PIB.csv', sep = ';', index = False)

In [2]:
import pandas as pd

In [4]:
df_model_vf = pd.read_csv('./Files/dataset_modelo.csv', sep = ';')

In [5]:
df_model_vf.head()

,Year,HHId,HHHolidays,HHFood,HHReserves,HHPhone,HHTV,HHComputer,HHWashMachine,HHCar,...,AREMonth_D_EM_SW_Dificult,AREMonth_D_EM_SW_Easy,AREMonth_D_EM_VeryEasy,AREMonth_D_EM_Verydificult,AREMonth_D_Unknown / Not Declared,LifeSatisfaction0,LifeSatisfaction1,LifeSatisfaction2,LifeSatisfaction_int,Weight
0,2013,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,0,0,0,0,0,8.0,19.241614,8.000000,8,1783.260169
1,2013,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,0,0,0,0,0,8.0,19.241614,8.000000,8,1783.260169
2,2013,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,0,0,0,0,0,6.5,16.182607,6.728170,6,1783.260169
3,2013,2,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,1,0,0,0,0,8.5,20.540864,8.540183,8,1783.260169
4,2013,2,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,1,0,0,0,0,7.5,19.233254,7.996524,7,1783.260169


In [8]:
list(df_model_vf.columns)

['Year',
 'HHId',
 'HHHolidays',
 'HHFood',
 'HHReserves',
 'HHPhone',
 'HHTV',
 'HHComputer',
 'HHWashMachine',
 'HHCar',
 'AREMonth',
 'HousingCost',
 'HHHeath',
 'HHnMembers',
 'HHConsUnits',
 'vhRentaa',
 'vhRentaAIa',
 'vhPobreza',
 'vhMATDEP',
 'Year_HHid',
 'Media_renta',
 'Std_renta',
 'vhRentaa_zscore',
 'vhRentaa_xperson',
 'Media_renta_ucons',
 'Std_renta_ucons',
 'vhRentaa_uncons_zscore',
 'Media_renta2',
 'Std_renta2',
 'vhRentaa2_zscore',
 'vhRentaa_xperson2',
 'Media_renta_ucons2',
 'Std_renta_ucons2',
 'vhRentaa_uncons2_zscore',
 'AREMonth_D',
 'Region',
 'Habitat',
 'Comunidad Autónoma_x',
 'year_x',
 'PIB_',
 'Pob_',
 'PIB_percapita_',
 'PIB(%)_change_',
 'PIB(%)_varnacional_',
 'Comunidad Autónoma_y',
 'year_y',
 'PIB_Nacional',
 'Pob_Nacional',
 'PIB_percapita_Nacional',
 'PIB(%)_change_Nacional',
 'IndID',
 'Sex',
 'YBirht',
 'MarStatus',
 'WorStatus',
 'FStudies',
 'CHealth',
 'CrConditions',
 'HLimitations',
 'MDClothes',
 'MDShoes',
 'MDFriends',
 'MDLeisure',
 

In [9]:
df_model_vf = df_model_vf[['vhRentaa','HousingCost_HighImpactHH','CrConditions_YChronic','HLimitations_SerLimited', 'MDInternet_Yes',
 'MDSelf_Yes', 'MDLeisure_Yes',  'MDFriends_Yes', 'MDShoes_Yes', 'MDClothes_Yes','CHealth','AREMonth','LifeSatisfaction2',
 'Year', 'LifeSatisfaction0','Weight','vhRentaa_xperson']]

In [10]:
df_model_vf.to_csv('')

,vhRentaa,HousingCost_HighImpactHH,CrConditions_YChronic,HLimitations_SerLimited,MDInternet_Yes,MDSelf_Yes,MDLeisure_Yes,MDFriends_Yes,MDShoes_Yes,MDClothes_Yes,CHealth,AREMonth,LifeSatisfaction2,Year,LifeSatisfaction0,Weight,vhRentaa_xperson
0,88450.00,1,0,0,1,1,1,1,1,1,1.0,5.0,8.000000,2013,8.0,1783.260169,38456.521739
1,88450.00,1,0,0,1,1,1,1,1,1,2.0,5.0,8.000000,2013,8.0,1783.260169,38456.521739
2,88450.00,1,0,0,1,1,0,1,1,1,2.0,5.0,6.728170,2013,6.5,1783.260169,38456.521739
3,30558.69,0,0,0,1,1,1,1,1,0,1.0,3.0,8.540183,2013,8.5,1783.260169,16977.050000
4,30558.69,0,0,0,1,1,1,1,1,1,2.0,3.0,7.996524,2013,7.5,1783.260169,16977.050000
